# Socioviews

### Imports

In [2]:
from main import initApp
import pandas as pd
import numpy as np
import os

## Preprocessament de dades

### Carregar les dades

In [3]:
resultat_path = os.path.join('dades', 'resultat')

#### Dades demogràfiques

In [4]:
# Llegir fitxer
demografica = pd.read_csv(os.path.join(resultat_path, 'demografica.csv'))

# Eliminar columnes innecessàries
demografica.drop(columns="Unnamed: 0", inplace=True)

# Canviar nom columna "Literal"
demografica.rename({"Literal": "Municipi"}, axis=1, inplace=True)

# Imprimir info
print("Dimensió demogràfica")
print(demografica.info())

Dimensió demogràfica
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19901 entries, 0 to 19900
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Codi                       19901 non-null  int64  
 1   Municipi                   19901 non-null  object 
 2   Any                        19901 non-null  int64  
 3   Total                      19901 non-null  int64  
 4   Homes                      19901 non-null  int64  
 5   Dones                      19901 non-null  int64  
 6   0-14 Anys                  19901 non-null  int64  
 7   15-64 Anys                 19901 non-null  int64  
 8   64 o més                   19901 non-null  int64  
 9   Índex d'envelliment        500 non-null    float64
 10  Índex de sobreenvelliment  500 non-null    float64
 11  Saldo migratori total      17988 non-null  float64
 12  Immigracions               17988 non-null  float64
 13  Emigracions              

#### Dades econòmiques

In [5]:
# Llegir fitxer
economica = pd.read_csv(os.path.join(resultat_path, 'economica.csv'))

# Eliminar columnes innecessàries
economica.drop(columns="Unnamed: 0", inplace=True)

# Canviar nom columna "Literal"
economica.rename({"Literal": "Municipi"}, axis=1, inplace=True)

# Imprimir info
print("\nDimensió econòmica")
print(economica.info())


Dimensió econòmica
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9824 entries, 0 to 9823
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Municipi         9824 non-null   object 
 1   Codi             9824 non-null   int64  
 2   Any              9824 non-null   int64  
 3   Aturats          8523 non-null   float64
 4   Evolucio atur    7576 non-null   float64
 5   Poblacio activa  762 non-null    float64
 6   Renda            4086 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 537.4+ KB
None


#### Dades residencials

In [6]:
# Llegir fitxer
residencial = pd.read_csv(os.path.join(resultat_path, 'residencial.csv'))

# Eliminar columnes innecessàries
residencial.drop(columns="Unnamed: 0", inplace=True)

# Canviar nom columna "Literal"
residencial.rename({"Literal": "Municipi"}, axis=1, inplace=True)

# Imprimir info
print("\nDimensió residencial")
print(residencial.info())


Dimensió residencial
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5069 entries, 0 to 5068
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Municipi                         5069 non-null   object 
 1   Codi                             5069 non-null   int64  
 2   Any                              5069 non-null   int64  
 3   Demanda                          3408 non-null   float64
 4   Habitatges familiars secundaris  1893 non-null   float64
 5   Habitatges familiars vacants     1893 non-null   float64
 6   Habitatges familiars altres      946 non-null    float64
 7   Habitatges familiars principals  1893 non-null   float64
 8   De lloguer                       1411 non-null   float64
 9   De propietat                     1411 non-null   float64
dtypes: float64(7), int64(2), object(1)
memory usage: 396.1+ KB
None


#### Dades urbanístiques

In [7]:
# Llegir fitxer
urbanistica = pd.read_csv(os.path.join(resultat_path, 'urbanistica.csv'))

# Eliminar columnes innecessàries
urbanistica.drop(columns="Unnamed: 0", inplace=True)

# Canviar nom columna "Literal"
urbanistica.rename({"Literal": "Municipi"}, axis=1, inplace=True)

# Imprimir info
print("\nDimensió urbanística")
print(urbanistica.info())


Dimensió urbanística
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8526 entries, 0 to 8525
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Any                 8526 non-null   int64  
 1   Serveis             8523 non-null   float64
 2   Municipi            8526 non-null   object 
 3   Codi                8526 non-null   int64  
 4   Solar               8523 non-null   float64
 5   Zones verdes        8523 non-null   float64
 6   Equipament          8523 non-null   float64
 7   Connectivitat       8523 non-null   float64
 8   Densitat comercial  2841 non-null   float64
 9   Riscos              814 non-null    float64
dtypes: float64(7), int64(2), object(1)
memory usage: 666.2+ KB
None


#### Dades històriques habitatge social

In [8]:
# Llegir fitxer
habitatges = pd.read_excel(os.path.join('dades', 'habitatge_social.xlsx'))

# Eliminar columnes innecessàries
habitatges.drop(columns=["NOM_MUNICIP", "Data Construcció", "Total Hab"], inplace=True)

# Imprimir info
print(habitatges.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Municipi     238 non-null    object
 1   Any          238 non-null    int64 
 2   Hab Lloguer  238 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.7+ KB
None


#### Municipis de Catalunya

In [10]:
# Llegir fitxer
municipis = pd.read_csv(os.path.join('dades', 'noms_municipis.csv'))

# Eliminar columnes innecessàries
municipis.drop(columns="NomMun", inplace=True)

# Imprimir info
print(municipis.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Municipi  947 non-null    object
 1   Codi      947 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.9+ KB
None


### Combinar dades

#### Creació d'un dataframe amb una file per any i municipi

In [11]:
# Anys a realitzar l'estudi 
years = range(2000, 2021)

# Creació dataframe
municipis = pd.concat([municipis.assign(Any = y) for y in years], ignore_index=True)

# Imprimir info
print(municipis.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19887 entries, 0 to 19886
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Municipi  19887 non-null  object
 1   Codi      19887 non-null  int64 
 2   Any       19887 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 466.2+ KB
None


#### Combinar municipis amb dades indicadors socioeconòmics 

In [13]:
# Demogràfica
indicadors = municipis.merge(demografica, how='left', on=["Codi", "Municipi", "Any"])
print(indicadors.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19887 entries, 0 to 19886
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Municipi                   19887 non-null  object 
 1   Codi                       19887 non-null  int64  
 2   Any                        19887 non-null  int64  
 3   Total                      19775 non-null  float64
 4   Homes                      19775 non-null  float64
 5   Dones                      19775 non-null  float64
 6   0-14 Anys                  19775 non-null  float64
 7   15-64 Anys                 19775 non-null  float64
 8   64 o més                   19775 non-null  float64
 9   Índex d'envelliment        500 non-null    float64
 10  Índex de sobreenvelliment  500 non-null    float64
 11  Saldo migratori total      17893 non-null  float64
 12  Immigracions               17893 non-null  float64
 13  Emigracions                17893 non-null  flo

In [14]:
# Economica
indicadors = indicadors.merge(economica, how='left', on=["Codi", "Municipi", "Any"])
print(indicadors.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19887 entries, 0 to 19886
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Municipi                   19887 non-null  object 
 1   Codi                       19887 non-null  int64  
 2   Any                        19887 non-null  int64  
 3   Total                      19775 non-null  float64
 4   Homes                      19775 non-null  float64
 5   Dones                      19775 non-null  float64
 6   0-14 Anys                  19775 non-null  float64
 7   15-64 Anys                 19775 non-null  float64
 8   64 o més                   19775 non-null  float64
 9   Índex d'envelliment        500 non-null    float64
 10  Índex de sobreenvelliment  500 non-null    float64
 11  Saldo migratori total      17893 non-null  float64
 12  Immigracions               17893 non-null  float64
 13  Emigracions                17893 non-null  flo

In [15]:
# Residencial
indicadors = indicadors.merge(residencial, how='left', on=["Codi", "Municipi", "Any"])
print(indicadors.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19887 entries, 0 to 19886
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Municipi                         19887 non-null  object 
 1   Codi                             19887 non-null  int64  
 2   Any                              19887 non-null  int64  
 3   Total                            19775 non-null  float64
 4   Homes                            19775 non-null  float64
 5   Dones                            19775 non-null  float64
 6   0-14 Anys                        19775 non-null  float64
 7   15-64 Anys                       19775 non-null  float64
 8   64 o més                         19775 non-null  float64
 9   Índex d'envelliment              500 non-null    float64
 10  Índex de sobreenvelliment        500 non-null    float64
 11  Saldo migratori total            17893 non-null  float64
 12  Immigracions      

In [16]:
# Urbanistica
indicadors = indicadors.merge(urbanistica, how='left', on=["Codi", "Municipi", "Any"])
print(indicadors.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19887 entries, 0 to 19886
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Municipi                         19887 non-null  object 
 1   Codi                             19887 non-null  int64  
 2   Any                              19887 non-null  int64  
 3   Total                            19775 non-null  float64
 4   Homes                            19775 non-null  float64
 5   Dones                            19775 non-null  float64
 6   0-14 Anys                        19775 non-null  float64
 7   15-64 Anys                       19775 non-null  float64
 8   64 o més                         19775 non-null  float64
 9   Índex d'envelliment              500 non-null    float64
 10  Índex de sobreenvelliment        500 non-null    float64
 11  Saldo migratori total            17893 non-null  float64
 12  Immigracions      